In [1]:
from src.dysregnet import dysregnet
import pandas as pd
import pickle
import numpy as np


Get data

In [2]:
grn_breast = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/GENIE3_output/linkedList_output_slurm_ gene_tpm_v10_breast_mammary_tissu_filtered .csv")
grn_lung = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/GENIE3_output/linkedList_output_slurm_ gene_tpm_v10_lung_filtered .csv")

exp_lung = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/gtex/gene_tpm_v10_lung_filtered.csv")
exp_breast = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/dysregnet_gtex/gtex/gene_tpm_v10_breast_mammary_tissu_filtered.csv")

In [3]:
meta_lung = pd.DataFrame({
    'sample': exp_lung.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_lung.columns[1:])),
    'gender' : 'FEMALE'
})
meta_lung

,sample,condition,gender
0,GTEX-111CU-0326-SM-5GZXO,1,FEMALE
1,GTEX-111FC-1126-SM-5GZWU,1,FEMALE
2,GTEX-111VG-0726-SM-5GIDC,0,FEMALE
3,GTEX-111YS-0626-SM-5GZXV,0,FEMALE
4,GTEX-1122O-0126-SM-5GICA,1,FEMALE
...,...,...,...
599,GTEX-ZYVF-1726-SM-5E443,1,FEMALE
600,GTEX-ZYW4-1526-SM-5MR8B,0,FEMALE
601,GTEX-ZYY3-0926-SM-5E454,1,FEMALE
602,GTEX-ZZPT-1326-SM-5E43H,0,FEMALE


In [4]:
grn_breast
meta_breast = pd.DataFrame({
    'sample': exp_breast.columns[1:],
    'condition': np.random.choice([0, 1], size=len(exp_breast.columns[1:])),
    'gender' : 'FEMALE'
})
meta_breast

,sample,condition,gender
0,GTEX-1117F-2826-SM-5GZXL,1,FEMALE
1,GTEX-111YS-1926-SM-5GICC,1,FEMALE
2,GTEX-1122O-1226-SM-5H113,1,FEMALE
3,GTEX-117XS-1926-SM-5GICO,0,FEMALE
4,GTEX-117YX-1426-SM-5H12H,1,FEMALE
...,...,...,...
509,GTEX-ZYT6-0126-SM-5E45J,1,FEMALE
510,GTEX-ZYW4-0826-SM-5GIDG,0,FEMALE
511,GTEX-ZYY3-2526-SM-GMXAZ,0,FEMALE
512,GTEX-ZZ64-1226-SM-5E43R,1,FEMALE


In [5]:
grn_lung

,regulatoryGene,targetGene,weight
0,ZFY,TTTY14,0.269712
1,ZFY,EIF1AY,0.248673
2,ZFY,UTY,0.244710
3,ZFY,TXLNGY,0.240938
4,ZFY,USP9Y,0.239011
...,...,...,...
99995,ZNF571,PRPSAP1,0.014408
99996,L3MBTL1,CERS3-AS1,0.014408
99997,TWIST1,DIO2,0.014408
99998,DMRTA1,MAGI3,0.014408


Define the confounding variables or the design matrix

In [6]:
# define condition column (0 indicated control, 1 indicates case), Column name for the condition in the meta DataFrame.
conCol='condition'

# define categorical confounder columns in meta dataframe, List of categorical variable names. They should match the name of their columns in the meta Dataframe.
CatCov=['gender']  

# define continuous confounder columns in meta dataframe, List of continuous covariates. They should match the name of their columns in the meta Dataframe.
# ConCov=['gender']

In [7]:
exp_lung = exp_lung.set_index(exp_lung.columns[0])
exp_lung = exp_lung.T 
exp_lung.insert(0, "sample", exp_lung. index)
assert all(exp_lung.iloc[:, 0].values == meta_lung.iloc[:,0].values)

In [8]:
exp_breast = exp_breast.set_index(exp_breast.columns[0])
exp_breast = exp_breast.T 
exp_breast.insert(0, "sample", exp_breast. index)
assert all(exp_breast.iloc[:, 0].values == meta_breast.iloc[:,0].values)

Run DysRegNet

In [9]:
data=dysregnet.run(expression_data=exp_breast,
                   meta=meta_breast, 
                   GRN="breast",
                   conCol=conCol,
                   CatCov=CatCov,
                   direction_condition=False,
                   normaltest=True,
                   R2_threshold=.2)

Processing breast GRN file...
GRN file and model ZIP already exist locally. Skipping download.
Extracting breast model ZIP...
Extracted models/breast_models.zip to models


Processing edges: 100000it [02:14, 741.87it/s]


Ratio of found models:  1.0
Skipped models:  0.0


In [10]:
# Load the pickle file
with open("pickle_models/AHR_FOS.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [11]:
print("Attributes and methods of the loaded object:")
print(dir(loaded_model))

Attributes and methods of the loaded object:
['HC0_se', 'HC1_se', 'HC2_se', 'HC3_se', '_HCCM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abat_diagonal', '_cache', '_data_attr', '_data_in_cache', '_get_robustcov_results', '_get_wald_nonlinear', '_is_nested', '_transform_predict_exog', '_use_t', '_wexog_singular_values', 'aic', 'bic', 'bse', 'centered_tss', 'compare_f_test', 'compare_lm_test', 'compare_lr_test', 'condition_number', 'conf_int', 'conf_int_el', 'cov_HC0', 'cov_HC1', 'cov_HC2', 'cov_HC3', 'cov_kwds', 'cov_params', 'cov_type', 'df_model', 'df_resid', 'eigenvals', 'el_test', 'ess', 'f_pvalue', 'f_test', 'fittedvalues', 'fvalue', 'get_influence', 'get_prediction',

In [12]:
print("Variables and their values in the loaded object:")
print(vars(loaded_model))

Variables and their values in the loaded object:
{'_results': <statsmodels.regression.linear_model.OLSResults object at 0x1774e52d0>, '__doc__': "\n    Results class for for an OLS model.\n\n    Parameters\n    ----------\n    model : RegressionModel\n        The regression model instance.\n    params : ndarray\n        The estimated parameters.\n    normalized_cov_params : ndarray\n        The normalized covariance parameters.\n    scale : float\n        The estimated scale of the residuals.\n    cov_type : str\n        The covariance estimator used in the results.\n    cov_kwds : dict\n        Additional keywords used in the covariance specification.\n    use_t : bool\n        Flag indicating to use the Student's t in inference.\n    **kwargs\n        Additional keyword arguments used to initialize the results.\n\n    See Also\n    --------\n    RegressionResults\n        Results store for WLS and GLW models.\n\n    Notes\n    -----\n    Most of the methods and attributes are inherit

In [13]:
print(loaded_model.params)  # Coefficients
print(loaded_model.summary())  # Full model summary

const          0.000360
AHR            0.323355
gender_MALE   -0.000850
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     3.268
Date:                Wed, 15 Jan 2025   Prob (F-statistic):             0.0454
Time:                        19:20:30   Log-Likelihood:                -80.461
No. Observations:                  59   AIC:                             166.9
Df Residuals:                      56   BIC:                             173.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

In [14]:
# get the patient-specific dysregulate networks
# results = data.get_results()

# or with binary edges
results = data.get_results_binary()

# get R2 values, coefficients, and coefficient p-values for all models/edges
# data.get_model_stats()

In [15]:
edge_counts = results.sum()
most_frequent_edge = edge_counts.idxmax()
max_count = edge_counts.max()

print (f"The most frequent edge is {most_frequent_edge} with {max_count} occurrences.")

The most frequent edge is ('FOXA1', 'DRAIC') with 8.0 occurrences.
